In [8]:
import gurobipy as gp 
from gurobipy import GRB

In [3]:
import numpy as np
import math
def randInstanceGen(n,m,J,U):
    # Reset seed for safety
    np.random.seed(0)
    # Set seed
    np.random.seed(61836)
    
    c = np.random.randint(1,U+1,(J,n))
    a = np.random.randint(1,U+1,(n,m))
    
    b = np.max(a, axis=0)
    
    for i in range(0, m):
        if math.ceil(np.sum(a[:,i])/2) > b[i]:
            b[i] = math.ceil(np.sum(a[:,i])/2)
    
    return c, a, b

In [4]:
c, a, b = randInstanceGen(10,1,2,40)
c[1] = c[1]*-1
print(c)
print("**")
print(a)
print("**")
print(b)

[[ 34   1  37  17   1  40   3  32  31   7]
 [-29 -22 -12 -40 -20  -2 -23  -7  -7 -31]]
**
[[10]
 [23]
 [ 6]
 [35]
 [24]
 [20]
 [16]
 [24]
 [15]
 [38]]
**
[106]


In [ ]:
def rectangular(c,a,b):
    found = []
    
    m = gp.Model("Rectangular Method")
    
    n = a.shape[1]
    cons = b.shape[0]
    

    Items=list(np.arange(0,n))
    Dimensions= list(np.arange(0,cons))

    # Decision Vars
    x = m.addVars(n,vtype=GRB.BINARY, name="ToTakeOrNotToTake")

    #Mute Output Text
    m.Params.OutputFlag = 0


    #Knapsack Constraints
    for j in Dimensions:
        m.addConstr(gp.quicksum(x[i]*a[i,j] for i in Items) <= b[j], name="j"+str(j))

    #NW 
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    NW = (lexConstraint,hold) 
    
    #SE 
    old = m.getConstrByName('temp')
    m.remove(old)
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    SE = (hold, lexConstraint) 
    
    old = m.getConstrByName('temp')
    m.remove(old)
    
    rectangles = [[NW,SE]]
    
    #points in () , tuples
    #rectangles 2D lists of points 
    
    while len(rectangles) != 0:
        
        R = rectangles.pop(0)
     
        
        R2 = [(R[0][0] , (R[0][1]+R[1][1])/2.0), R[1] ] 
        
        
        #lexmin(z1,z2)  (going left)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R2[0][1] , name = 'top bound')
        #m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R2[0][1] , name = 'region')
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        
        m.optimize()
        z1 = m.objVal
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = m.objVal
        tempNW = (z1,hold) 
        
        if tempNW != R[1]:
            found.append(tempNW)
            rectangles.append([tempNW, R[1]])
            
        R3 = [(R[0],    ((tempNW[0] - e)/2, (R[0][1] + R[1][1])/2)  ]    
               
               
        #lexmin(z2,z1) = z_opt 
        #Do i need an extra constraint here???  bottom bound??     
        old = m.getConstrByName('temp')
        m.remove(old)       
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        
         
        #if z_opt =! R3[0] 
         #found.append(z_opt)
         #rectangles.add [R3[0] , Z_opt ]
        
               
               
               
    return found 
    
    
    
    
    
    